<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LSTM - Clase 5 - V4.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [9]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data_full.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [10]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [11]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64


In [ ]:
top_product_ids = data.groupby('product_id')['tn'].sum().sort_values(ascending=False).head(25).index.tolist()

In [ ]:
data = data[data['product_id'].isin(top_product_ids)]

In [ ]:
scaler = MinMaxScaler()
X = []
y = []

for product_id in top_product_ids:
    product_data = data[data['product_id'] == product_id]
    product_data = product_data.sort_values('periodo')
    
    # Normalizar los datos
    product_data['tn'] = scaler.fit_transform(product_data['tn'].values.reshape(-1, 1))
    
    # Crear secuencias de entrada y salida
    for i in range(len(product_data) - 1):
        X.append(product_data['tn'].values[i])
        y.append(product_data['tn'].values[i + 1])

X = np.array(X)
y = np.array(y)

# Dar forma a los datos para el modelo LSTM
X = X.reshape(X.shape[0], 1, 1)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X, y, epochs=100, batch_size=32, verbose=0)

In [ ]:
# Crear una lista con los product_id más importantes
product_ids = top_product_ids

# Crear un DataFrame vacío para las predicciones
predictions = pd.DataFrame(columns=['product_id', 'tn'])

# Hacer predicciones para cada product_id
for product_id in product_ids:
    product_data = data[data['product_id'] == product_id]
    product_data = product_data.sort_values('periodo')
    
    # Obtener el último valor de tn
    last_tn = product_data['tn'].values[-1]
    
    # Hacer la predicción para febrero de 2020
    prediction = scaler.inverse_transform(model.predict(np.array([last_tn]).reshape(1, 1, 1)))[0][0]
    
    # Agregar la predicción al DataFrame
    predictions = predictions.append({'product_id': product_id, 'tn': prediction}, ignore_index=True)

# Guardar el DataFrame de predicciones en un archivo CSV
predictions.to_csv('LSTM_top25.v5.0.1.csv', index=False)

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 